In [1]:
import sys
import gym
import numpy as np
from time import sleep
import random
import tqdm

In [2]:
def create_random_policy(env):
    policy = {}
    for key in range(0, env.observation_space.n):
        current_end = 0
        p = {}
        for action in range(0, env.action_space.n):
            p[action] = 1 / env.action_space.n
        policy[key] = p
    return policy

def create_state_action_dictionary(env, policy):
    Q = {}
    for key in policy.keys():
         Q[key] = {a: 0.0 for a in range(0, env.action_space.n)}
    return Q

In [5]:
def run_game(env, policy, display=True):
    env.reset()
    episode = []
    finished = False

    while not finished:
        s = env.env.s
        if display:
            clear_output(True)
            env.render()
            sleep(0.2)

        timestep = []
        timestep.append(s)
        n = random.uniform(0, sum(policy[s].values()))
        top_range = 0
        for prob in policy[s].items():
                top_range += prob[1]
                if n < top_range:
                    action = prob[0]
                    break 
        state, reward, finished, info = env.step(action)
        timestep.append(action)
        timestep.append(reward)

        episode.append(timestep)

    if display:
        clear_output(True)
        env.render()
        sleep(0.2)
    return episode

def test_policy(policy, env):
    wins = 0
    r = 100
    for i in range(r):
        w = run_game(env, policy, display=False)[-1][-1]
        if w == 1:
                wins += 1
    return wins / r

In [6]:
def monte_carlo(env, episodes=100, policy=None, epsilon=0.01, first_visit=True):
    if not policy:
        policy = create_random_policy(env)  # Create an empty dictionary to store state action values    
    Q = create_state_action_dictionary(env, policy) # Empty dictionary for storing rewards for each state-action pair
    returns = {}
    
    for idx in range(episodes): # Looping through episodes
        G = 0 # Store cumulative reward in G (initialized at 0)
        episode = run_game(env=env, policy=policy, display=False) # Store state, action and value respectively 
        
        # for loop through reversed indices of episode array. 
        # The logic behind it being reversed is that the eventual reward would be at the end. 
        # So we have to go back from the last timestep to the first one propagating result from the future.
        
        for i in reversed(range(0, len(episode))):   
            s_t, a_t, r_t = episode[i] 
            state_action = (s_t, a_t)
            G += r_t # Increment total reward by reward on current timestep
            
            # filter for first-visit state-action pair only
            if first_visit:
                if state_action in [(x[0], x[1]) for x in episode[0:i]]:
                    continue
                
            if returns.get(state_action):  # store return value of a state-action pair
                returns[state_action].append(G)
            else:
                returns[state_action] = [G]   
                
            # update Q value for a state-action pair
            Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
            
            # finding the action with maximum value
            Q_list = list(map(lambda x: x[1], Q[s_t].items()))
            indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
            max_Q = random.choice(indices)
            
            # update action probability for s_t in policy
            for action, prob in policy[s_t].items():
                if action == max_Q:
                    policy[s_t][action] = 1 - epsilon + (epsilon / abs(sum(policy[s_t].values())))
                else:
                    policy[s_t][action] = (epsilon / abs(sum(policy[s_t].values())))
        
        if (idx+1) % 5000 == 0:
            print("\rEpisode {}/{}.".format(idx+1, episodes), end="")
            sys.stdout.flush()

    return Q, policy

In [7]:
env = gym.make('FrozenLake-v1', is_slippery=False)
Q, policy = monte_carlo(env, episodes=100000, first_visit=True)
test_policy(policy, env)

Episode 100000/100000.

0.97

In [88]:
policy

{0: {0: 0.00971933168334964,
  1: 0.00971933168334964,
  2: 0.9997193316833496,
  3: 0.00971933168334964},
 1: {0: 0.00971933168334964,
  1: 0.00971933168334964,
  2: 0.9997193316833496,
  3: 0.00971933168334964},
 2: {0: 0.00971933168334964,
  1: 0.9997193316833496,
  2: 0.00971933168334964,
  3: 0.00971933168334964},
 3: {0: 0.9997193316833496,
  1: 0.00971933168334964,
  2: 0.00971933168334964,
  3: 0.00971933168334964},
 4: {0: 0.00971933168334964,
  1: 0.00971933168334964,
  2: 0.00971933168334964,
  3: 0.9997193316833496},
 5: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25},
 6: {0: 0.00971933168334964,
  1: 0.9997193316833496,
  2: 0.00971933168334964,
  3: 0.00971933168334964},
 7: {0: 0.25, 1: 0.25, 2: 0.25, 3: 0.25},
 8: {0: 0.00971933168334964,
  1: 0.00971933168334964,
  2: 0.9997193316833496,
  3: 0.00971933168334964},
 9: {0: 0.00971933168334964,
  1: 0.00971933168334964,
  2: 0.9997193316833496,
  3: 0.00971933168334964},
 10: {0: 0.00971933168334964,
  1: 0.9997193316833496,
  2: 

In [89]:
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG
